In [1]:
from __future__ import absolute_import, division, print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

learning_rate = 0.0001
epochs = 120
display_step = 10
batch_size = 512
num_input = 784
num_classe = 10
dropout = 0.20

X = tf.placeholder("float32", [None, num_input])
Y = tf.placeholder("float32", [None, num_classe])

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Criação do modelo CNN

In [3]:
def cnn_model(x, dropout = 0.2, reuse = False):
    
    with tf.variable_scope("Conv", reuse = not reuse):
        x = tf.reshape(x, [-1, 28, 28, 1])
        
        conv1 = tf.layers.conv2d(x, filters=32, padding="SAME", 
                            kernel_size=[5,5], kernel_initializer=tf.truncated_normal_initializer(stddev=0.2),
                                activation=tf.nn.relu, name="conv_1")
        pool_conv1 = tf.layers.max_pooling2d(conv1, pool_size=[2,2], strides=[2,2], name="pool_1")
        
        conv2 = tf.layers.conv2d(pool_conv1, filters=64, padding="SAME",
                            kernel_size=[5,5], kernel_initializer=tf.truncated_normal_initializer(stddev=0.2),
                                activation=tf.nn.relu, name="conv_2")      
        pool_conv2 = tf.layers.max_pooling2d(conv2, pool_size=[2,2], strides=[2,2], name="pool_2")

        pool2_flat = tf.reshape(pool_conv2, [-1, 7 * 7 * 64])
        dense = tf.layers.dense(pool2_flat, units=1024, activation=tf.nn.relu)
        dropout = tf.layers.dropout(inputs=dense, rate=dropout)
        
        logits = tf.layers.dense(dropout, num_classe)       
        out = tf.nn.softmax(logits)

        return out, logits

## Inicialização do modelo, calculo do loss e accuracy

In [4]:
out, logits = cnn_model(X, reuse=True)

#Loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
train_op = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

#Accuracy
correct_prediction = tf.equal(tf.argmax(out, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use keras.layers.dropout instead.


In [5]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1, epochs+1):
        # Prepare Data
        # Get the next batch of MNIST data (only images are needed, not labels)
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        losses_train = []
        
        # Run optimization op (backprop) and cost op (to get loss value)
        sess.run([train_op, loss, accuracy], feed_dict={X:batch_x, Y:batch_y})
        
        _, l, acc = sess.run([train_op, loss, accuracy], feed_dict={X: batch_x, Y:batch_y})
        
        losses_train.append(l)

        if i % display_step == 0 or i == 1:
            print(' Train Step %i: Minibatch Loss: %f       accuracy: %f' % (i, l, acc))
            
        batch_x, batch_y = mnist.test.next_batch(batch_size)
        losses_test = []  
        l, acc = sess.run([loss, accuracy], feed_dict={X: batch_x, Y: batch_y})
        
        losses_test.append(l)
        
        if i % display_step == 0 or i == 1:
            print(' Test Step %i: Minibatch Loss: %f       accuracy: %f' % (i, l, acc))
            print() 
            
        #sample = sess.run(cnn_model(X), feed_dict={X:batch_x, Y:batch_y})

 Train Step 1: Minibatch Loss: 2.884248       accuracy: 0.097656
 Test Step 1: Minibatch Loss: 2.815013       accuracy: 0.087891

 Train Step 10: Minibatch Loss: 2.496011       accuracy: 0.138672
 Test Step 10: Minibatch Loss: 2.446350       accuracy: 0.132812

 Train Step 20: Minibatch Loss: 2.102013       accuracy: 0.224609
 Test Step 20: Minibatch Loss: 2.049023       accuracy: 0.261719

 Train Step 30: Minibatch Loss: 1.510066       accuracy: 0.609375
 Test Step 30: Minibatch Loss: 1.459704       accuracy: 0.625000

 Train Step 40: Minibatch Loss: 0.900489       accuracy: 0.787109
 Test Step 40: Minibatch Loss: 0.819279       accuracy: 0.810547

 Train Step 50: Minibatch Loss: 0.588410       accuracy: 0.832031
 Test Step 50: Minibatch Loss: 0.576754       accuracy: 0.835938

 Train Step 60: Minibatch Loss: 0.467116       accuracy: 0.845703
 Test Step 60: Minibatch Loss: 0.668316       accuracy: 0.806641

 Train Step 70: Minibatch Loss: 0.451431       accuracy: 0.869141
 Test Step 7